# Hierarchical model


In [22]:
import sympy
import mira
from mira.metamodel import *
from mira.modeling import Model
from mira.modeling.amr.petrinet import template_model_to_petrinet_json
from mira.sources.amr import model_from_json, model_from_url
from pyro.infer.inspect import get_dependencies
from pyciemss.compiled_dynamics import CompiledDynamics
from pyciemss.mira_integration.distributions import sort_mira_dependencies
import torch
import json
from chirho.dynamical.handlers.solver import TorchDiffEq
from pyciemss.compiled_dynamics import (
    _compile_param_values,
    get_name,
)
def test_acyclic_distribution_expressions():
     
    person_units = lambda: Unit(expression=sympy.Symbol('person'))
 
    beta_mean = Parameter(name='beta_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Float(1.0)*sympy.Symbol("gamma_mean"),
                                'beta': sympy.Float(10.0)}))
    gamma_mean = Parameter(name='gamma_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Float(10.0),
                                'beta': sympy.Float(10.0)}))
    beta = Parameter(name='beta',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    c = {
    "S": Concept(name="S", units=person_units(), identifiers={"ido": "0000514"}), # susceptible
    "I": Concept(name="I", units=person_units(), identifiers={"ido": "0000511"}), # infectious
    "R": Concept(name="R", units=person_units(), identifiers={"ido": "0000592"}), # recovered
    }


    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=c["S"],
                outcome=c['I'],
                controller=c['I'],
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=c['I'],
                outcome=c['R'],
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            },
        initials={'S': Initial(concept=c['S'], expression=sympy.Float(1.0)), 
                  'I': Initial(concept=c['I'], expression=sympy.Float(0.0)),
                  'R': Initial(concept=c['R'], expression=sympy.Float(0.0))}
    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('multilevel_sir_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean*gamma_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
        mira.metamodel.utils.SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')
    return sir_model

sort_mira_dependencies(test_acyclic_distribution_expressions())

['gamma_mean', 'gamma', 'beta_mean', 'beta']

In [21]:
model_from_url("https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/multilevel_sir_nodist_model.json")

TemplateModel(templates=[ControlledConversion(rate_law=I*S*beta, name='t1', display_name=None, type='ControlledConversion', controller=Concept(name='I', display_name='I', description=None, identifiers={}, context={}, units=None), subject=Concept(name='S', display_name='S', description=None, identifiers={}, context={}, units=None), outcome=Concept(name='I', display_name='I', description=None, identifiers={}, context={}, units=None), provenance=[]), NaturalConversion(rate_law=I*gamma, name='t2', display_name=None, type='NaturalConversion', subject=Concept(name='I', display_name='I', description=None, identifiers={}, context={}, units=None), outcome=Concept(name='R', display_name='R', description=None, identifiers={}, context={}, units=None), provenance=[])], parameters={'beta': Parameter(name='beta', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='InverseGamma1', parameters={'shape': beta_mean*gamma_mean, 'scale': 0

In [18]:
def test_acyclic_nondistribution_expressions():
    person_units = lambda: Unit(expression=sympy.Symbol('person'))

    beta_mean = Parameter(name='beta_mean',
                    value=1.0)
    gamma_mean = Parameter(name='gamma_mean',
                    value=2.0)
    beta = Parameter(name='beta',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    c = {
        "S": Concept(name="S", units=person_units(), identifiers={"ido": "0000514"}), # susceptible
        "I": Concept(name="I", units=person_units(), identifiers={"ido": "0000511"}), # infectious
        "R": Concept(name="R", units=person_units(), identifiers={"ido": "0000592"}), # recovered
    }

    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=Concept(name='S'),
                outcome=Concept(name='I'),
                controller=Concept(name='I'),
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=Concept(name='I'),
                outcome=Concept(name='R'),
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            },
        initials={'S': Initial(concept=c['S'], expression=sympy.Float(1.0)), 
                  'I': Initial(concept=c['I'], expression=sympy.Float(0.0)),
                  'R': Initial(concept=c['R'], expression=sympy.Float(0.0))}

    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('multilevel_sir_nodist_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean*gamma_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
        mira.metamodel.utils.SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')
    return sir_model

sort_mira_dependencies(test_acyclic_nondistribution_expressions())

['gamma_mean', 'beta_mean', 'gamma', 'beta']

In [13]:
from typing import Optional, Dict
import sympytorch

def safe_sympytorch_parse_expr(expr: SympyExprStr, local_dict: Optional[Dict]) -> torch.Tensor:
    """
    Converts a sympy expression to a PyTorch tensor.

    Parameters
    ----------
    expr : SympyExprStr
        The sympy expression to convert to a PyTorch tensor.
    local_dict : Optional[Dict]
        A dictionary of free symbols and their variables to use in the sympy expression."""
    return sympytorch.SymPyModule(expressions=[expr])(**local_dict).squeeze()

scale = safe_sympytorch_parse_expr(sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'), {'beta_mean': torch.tensor(1.0), 'gamma_mean': torch.tensor(2.0)})


tensor(2.)

In [ ]:
mod = sympytorch.SymPyModule(expressions=[sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')])

In [4]:
def test_beta_mean_cycle_distribution_expressions():
    beta_mean = Parameter(name='beta_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Float(1.0)*sympy.Symbol("beta_mean"),
                                'beta': sympy.Float(10.0)}))
    gamma_mean = Parameter(name='gamma_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Float(10.0),
                                'beta': sympy.Float(10.0)}))
    beta = Parameter(name='beta',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))

    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=Concept(name='S'),
                outcome=Concept(name='I'),
                controller=Concept(name='I'),
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=Concept(name='I'),
                outcome=Concept(name='R'),
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            }
    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('beta_mean_cycle_sir_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean*gamma_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
        mira.metamodel.utils.SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')
    return sir_model

sort_mira_dependencies(test_beta_mean_cycle_distribution_expressions())

NetworkXUnfeasible: Graph contains a cycle or graph changed during iteration

In [5]:
def test_gamma_mean_beta_mean_acyclic_distribution_expressions():
    beta_mean = Parameter(name='beta_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Integer(1)*sympy.Symbol("gamma_mean"),
                                'beta': sympy.Integer(10)}))
    gamma_mean = Parameter(name='gamma_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Integer(10)*sympy.Symbol("beta_mean"),
                                'beta': sympy.Integer(10)}))
    beta = Parameter(name='beta',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))

    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=Concept(name='S'),
                outcome=Concept(name='I'),
                controller=Concept(name='I'),
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=Concept(name='I'),
                outcome=Concept(name='R'),
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            }
    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('gamma_mean_beta_mean_cycle_sir_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean*gamma_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
        mira.metamodel.utils.SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')
    return sir_model

sort_mira_dependencies(test_gamma_mean_beta_mean_acyclic_distribution_expressions())

NetworkXUnfeasible: Graph contains a cycle or graph changed during iteration

In [6]:
def test_beta_mean_gamma_cycle_distribution_expressions():
    beta_mean = Parameter(name='beta_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Integer(10)*sympy.Symbol("gamma"),
                                'beta': sympy.Integer(10)}))
    gamma_mean = Parameter(name='gamma_mean',
                    distribution=Distribution(type="Beta1",
                    parameters={'alpha': sympy.Integer(10),
                                'beta': sympy.Integer(10)}))
    beta = Parameter(name='beta',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean'),
                                'scale': sympy.Float(0.01)}))
    gamma = Parameter(name='gamma',
                    distribution=Distribution(type="InverseGamma1",
                    parameters={'shape': sympy.Symbol('beta_mean'),
                                'scale': sympy.Float(0.01)}))

    # Make an SIR model with beta and gamma in rate laws
    sir_model = TemplateModel(
        templates=[
            ControlledConversion(
                subject=Concept(name='S'),
                outcome=Concept(name='I'),
                controller=Concept(name='I'),
                rate_law=sympy.Symbol('S') * sympy.Symbol('I') * sympy.Symbol('beta')
            ),
            NaturalConversion(
                subject=Concept(name='I'),
                outcome=Concept(name='R'),
                rate_law=sympy.Symbol('I') * sympy.Symbol('gamma')
            ),
        ],
        parameters={
            'beta': beta,
            'gamma': gamma,
            'beta_mean': beta_mean,
            'gamma_mean': gamma_mean,
            }
    )

    model = Model(sir_model)
    pn_json = template_model_to_petrinet_json(sir_model)
    with open('beta_mean_gamma_cycle_sir_model.json', 'w') as fh:
        json.dump(pn_json, fh)
        
    params = pn_json['semantics']['ode']['parameters']
    assert {p['id'] for p in params} == \
        {'beta_mean', 'gamma_mean', 'beta', 'gamma'}
    beta = [p for p in params if p['id'] == 'beta'][0]
    assert beta['distribution']['type'] == 'InverseGamma1'
    assert beta['distribution']['parameters']['shape'] == 'beta_mean*gamma_mean'

    # Now read the model back and check if it is deserialized
    tm = model_from_json(pn_json)
    assert tm.parameters['beta'].distribution.type == 'InverseGamma1'
    assert isinstance(tm.parameters['beta'].distribution.parameters['shape'],
        mira.metamodel.utils.SympyExprStr)
    assert tm.parameters['beta'].distribution.parameters['shape'].args[0] == \
        sympy.Symbol('beta_mean')*sympy.Symbol('gamma_mean')
    return sir_model

beta_mean_gamma_cycle = test_beta_mean_gamma_cycle_distribution_expressions()
sort_mira_dependencies(beta_mean_gamma_cycle)

model = model_from_json_file("beta_mean_gamma_cycle_sir_model.json")
sort_mira_dependencies(model)

NetworkXUnfeasible: Graph contains a cycle or graph changed during iteration

# Compile the non-hierarchical model and get dependencies

In [7]:
url_sirhd = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/sirhd.json"
compiled_model = CompiledDynamics.load(url_sirhd)
with TorchDiffEq():
    #simulation = compiled_model(start_time=torch.as_tensor(0.0), end_time=torch.as_tensor(0.0))
    dependencies = get_dependencies(compiled_model, model_args=[torch.as_tensor(0.0), torch.as_tensor(0.0)])
dependencies

{'prior_dependencies': {'persistent_beta': {'persistent_beta': set()},
  'persistent_gamma': {'persistent_gamma': set()}},
 'posterior_dependencies': {'persistent_beta': {'persistent_beta': set()},
  'persistent_gamma': {'persistent_gamma': set()}}}

# Compile the hierarchical model and get dependencies

In [ ]:
test_acyclic_distribution_expressions()


## Get all the mira parameters and their dependencies before we compile the dynamics

In [8]:
sir_model = test_acyclic_distribution_expressions()
sir_model.parameters

{'beta': Parameter(name='beta', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='InverseGamma1', parameters={'shape': beta_mean*gamma_mean, 'scale': 0.01})),
 'gamma': Parameter(name='gamma', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='InverseGamma1', parameters={'shape': gamma_mean, 'scale': 0.01})),
 'beta_mean': Parameter(name='beta_mean', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='Beta1', parameters={'alpha': gamma_mean, 'beta': 10})),
 'gamma_mean': Parameter(name='gamma_mean', display_name=None, description=None, identifiers={}, context={}, units=None, value=None, distribution=Distribution(type='Beta1', parameters={'alpha': 10, 'beta': 10}))}

## Get the dependencies in the mira model

First we go through each mira `Parameter` that is of type `Distribution`, and determine what other parameters it depends on based on whether the `Distribution.parameter` is a `SymPyExprStr` and the name of the parameter it depends on. Build a dependency DAG based on the this investigation, and return the DAG.

Then we topologically sort the DAG and call `MIRA_TO_PYRO` for each parameter in order.

In [20]:
import networkx as nx
def get_distribution_parameters(mira_dist: mira.metamodel.Distribution) -> list:
    return {k: v for k, v in mira_dist.parameters.items()}

def sort_mira_dependencies(src: TemplateModel, verbose=False) -> dict:
    dependencies = nx.DiGraph()

    for param_info in src.parameters.values():
        param_name = param_info.name
        param_dist = getattr(param_info, "distribution", None)
        if param_dist is None:
            param_value = param_info.value
        else:
            # Check to see if the distribution parameters are sympy expressions 
            for k, v in param_dist.parameters.items():
                if isinstance(v, mira.metamodel.utils.SympyExprStr):
                    if verbose:
                        print(f"Model parameter {param_name} has a sympy expression for distribution parameter {k} which has value {v}")
                    for free_symbol in v.free_symbols:
                        dependencies.add_edge(free_symbol, param_name)
                        if verbose:
                            print(f"    and free symbol {free_symbol}")
                elif verbose:
                    print(f"Model parameter {param_name} has a value for distribution parameter {k} which has value {v}")
    return list(nx.topological_sort(dependencies))



### Create a recursive mira distributions to pyro
However, I think we want to go one step above this so that we can determine the dependencies before we call MIRA_TO_PYRO.

## Inspect the hiearchical parameters

In [21]:
def get_name(param_info):
    return param_info.name

url_hierarchical = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/hierarchical_sir_model.json"
src = model_from_url(url_hierarchical)
mira_dependencies = get_mira_dependencies(sir_model)
#for param_info in src.parameters.values():
#    if isinstance(param_info, Parameter):
#        param_name = get_name(param_info)
        
mira_dependencies       


NameError: name 'get_mira_dependencies' is not defined

In [18]:
url_hierarchical = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/refs/heads/main/data/models/hierarchical_sir_model.json"
compiled_model = CompiledDynamics.load(url_hierarchical)
with TorchDiffEq():
    #simulation = compiled_model(start_time=torch.as_tensor(0.0), end_time=torch.as_tensor(0.0))
    dependencies = get_dependencies(compiled_model, model_args=[torch.as_tensor(0.0), torch.as_tensor(0.0)])
dependencies

/Users/zuck016/Projects/ASKEM/build/pyciemss/pyciemss/mira_integration/distributions.py:257: UserWarning: Conversion from MIRA distribution type InverseGamma1 to Pyro distribution has not been tested.
  warnings.warn(


ValueError: The model parameters could not be compiled. Please check the model definition.